In [2]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [49]:
df = pd.read_excel("datadownload.xlsx", skiprows=13)
def strip_date(x):
    x = x.replace("YE", "").replace("R", "").replace("P", "")
    x = x.strip()
    if x.startswith("YE"):
        x = x[2:].strip()
    return pd.to_datetime(x, format="%b %y")
df['date'] = df['Period'].apply(strip_date)
df = df.drop(columns=['Period'])
df = df.melt(id_vars=['date'], var_name='series', value_name='value')
df['label'] = np.where(df.date == df.date.max(), df.series, "")

events = [
    {
        "date": "2021-01-01",
        "label": "Introduction of\npoints-based immigration system",
    }
]

base = alt.Chart(df).encode(
    x=alt.X("date:T", title=""),
    y=alt.Y("value:Q", title=""),
    color=alt.Color("series:N", title="Series", legend=None, scale=alt.Scale(reverse=True)),
)

lines = base.mark_line().encode()

end_labels = base.mark_text(
    align='left',
    dx=5,
    fontSize=10,
    fontWeight='bold',
    color='black'
).encode(
    text=alt.Text("label:N")
).transform_filter(
    alt.datum.label != ""
)

zero_rule = alt.Chart(pd.DataFrame({
    "y": [0],
})).mark_rule(color='black', strokeDash=[1, 0]).encode(
    y=alt.Y("y:Q", title=""),
)

events_df = pd.DataFrame(events)
events_df['label'] = events_df['label'].apply(lambda x: x.split("\n"))

event_rules = alt.Chart(events_df).mark_rule(
    strokeDash=[3,5],
    opacity=0.8,
    color='#676A86').encode(
    x=alt.X("date:T", title=""),
)

event_label = alt.Chart(events_df).mark_text(
    align='left',
    fontSize=10,
    color='#676A86'
).encode(
    x=alt.X("date:T", title=""),
    y=alt.value(-30),
    text=alt.Text("label:N"),
)

chart = end_labels + lines + zero_rule + event_rules + event_label

chart = chart.properties(
    width=500,
    height=350
)

chart.save("chart.png", scale_factor=5)
chart.save("chart.svg", scale_factor=5)
chart.save("chart.json")
chart

alt.LayerChart(...)